In [3]:
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt

In [4]:
## write the function to create the column tema of the dataframe

def create_tema(stock: pd.DataFrame, MA_period:int ) -> pd.Series:

    ema1 = ta.ema(stock['Close'], length=MA_period)

    # Calculate the second EMA
    ema2 = ta.ema(ema1, length=MA_period)

    # Calculate the third EMA
    ema3 = ta.ema(ema2, length=MA_period)

    # Calculate TEMA
    tema = 3 * (ema1 - ema2) + ema3
    #stock[f'tema_{MA_period}'] = tema
    return tema

In [18]:
# Define your custom strategy using CustomStrategy
CustomStrategy = ta.Strategy(
    name="Customize technical indicator",
    description="adding TI",
    ta=[
        {"kind": "natr", 'length':3 },
        {"kind": "rsi", 'length':3 },
        {"kind": "adx", 'length':3 },
        {"kind": "cci", 'length':3 },
        {"kind": "roc", 'length':3 },
        {"kind": "stoch", 'length':3 },
        {"kind": "willr", 'length':3 },
        {"kind": "obv"},
        {"kind": "macd"},
        {"kind": "bbands", 'length':3},
    ]
)

In [19]:
def create_min_max_mid_price(stock: pd.DataFrame, period:int ) -> pd.Series:
    stock[f'min_price_{period}'] = stock['Low'].rolling(window=period).min()
    stock[f'max_price_{period}'] = stock['High'].rolling(window=period).max()
    stock['mid_price'] = (stock['High'] + stock['Low']) / 2

In [20]:
def calculate_MA_smoothness( ma:pd.Series):
    diff_i = ma -ma.shift(1)
    diff_i = diff_i.dropna()
    smooth =(diff_i-diff_i.shift(1)).dropna()
    smooth = smooth.abs().mean()
    return smooth
def calculate_MA_lag(stock: pd.DataFrame, ma:pd.Series):
    lag = (stock['Close']-ma).dropna()
    lag= lag.abs().mean()
    return lag

In [87]:
#import data csv and process the data
the_name_of_stock="msn"
data = pd.read_csv('/mnt/learning/last_project/fetch_data_stock/unprocessed_data/' + the_name_of_stock + '.csv')
data.head(10)

,Ngày,Lần cuối,Mở,Cao,Thấp,KL,% Thay đổi
0,21/07/2023,"81,800","79,500","81,900","79,500",1.75M,2.38%
1,20/07/2023,"79,900","80,900","80,900","79,100",2.20M,-1.11%
2,19/07/2023,"80,800","80,600","81,300","80,500",1.37M,0.37%
3,18/07/2023,"80,500","80,600","81,300","80,200",1.27M,-0.25%
4,17/07/2023,"80,700","81,800","81,800","80,500",1.45M,-1.34%
5,14/07/2023,"81,800","82,000","82,700","80,700",1.69M,0.00%
6,13/07/2023,"81,800","79,600","81,900","79,500",2.67M,2.89%
7,12/07/2023,"79,500","80,100","80,100","79,000",1.61M,0.38%
8,11/07/2023,"79,200","78,100","80,100","78,100",2.85M,1.67%
9,10/07/2023,"77,900","75,400","77,900","75,400",2.35M,3.59%


In [88]:
#change the column name from Vietnamese to English

data = data.rename(columns={'Ngày': 'Date', 'Mở': 'Open', 'Cao': 'High', 'Thấp': 'Low', 'Lần cuối': 'Close', 'KL': 'Volume'})

if '% Thay đổi' in data.columns:
    data = data.drop(columns=['% Thay đổi'])

data.head(10)

,Date,Close,Open,High,Low,Volume
0,21/07/2023,"81,800","79,500","81,900","79,500",1.75M
1,20/07/2023,"79,900","80,900","80,900","79,100",2.20M
2,19/07/2023,"80,800","80,600","81,300","80,500",1.37M
3,18/07/2023,"80,500","80,600","81,300","80,200",1.27M
4,17/07/2023,"80,700","81,800","81,800","80,500",1.45M
5,14/07/2023,"81,800","82,000","82,700","80,700",1.69M
6,13/07/2023,"81,800","79,600","81,900","79,500",2.67M
7,12/07/2023,"79,500","80,100","80,100","79,000",1.61M
8,11/07/2023,"79,200","78,100","80,100","78,100",2.85M
9,10/07/2023,"77,900","75,400","77,900","75,400",2.35M


In [89]:
#change the data type of the column Date to datetime and set index

data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data = data.set_index('Date')
data = data.sort_index()
main_df = data.copy()
main_df.head(10)

,Close,Open,High,Low,Volume
Date,,,,,
2013-07-29,"88,500","90,000","91,000","88,000",155.99K
2013-07-30,"89,000","88,000","89,500","88,000",156.25K
2013-07-31,"88,500","89,000","89,500","88,000",79.70K
2013-08-01,"88,000","88,500","88,500","88,000",273.93K
2013-08-02,"88,500","88,500","89,000","88,000",128.92K
2013-08-05,"88,000","88,500","89,000","88,000",87.00K
2013-08-06,"87,500","87,500","88,000","86,500",291.51K
2013-08-07,"88,000","87,500","88,500","87,000",200.23K
2013-08-08,"88,500","88,000","92,000","88,000",235.81K


In [90]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2492 entries, 2013-07-29 to 2023-07-21
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   2492 non-null   object
 1   Open    2492 non-null   object
 2   High    2492 non-null   object
 3   Low     2492 non-null   object
 4   Volume  2492 non-null   object
dtypes: object(5)
memory usage: 116.8+ KB


In [91]:
#Process the value of the column Volume to integer type and the value of the column Close, Open, High, Low to float type
#Using loop to process the data
#IF the volume value has character K remove it and ',' and multiply by 1000, if the volume value has character M remove it and ',' and multiply by 1000000
#Replace all character ',' in the value of the column Close, Open, High, Low to ''

for i in range(len(main_df['Volume'])):
    try:
        if 'K' in main_df['Volume'][i]:
            main_df['Volume'][i] = main_df['Volume'][i].replace('K', '')
            main_df['Volume'][i] = float(main_df['Volume'][i]) * 1000
        elif 'M' in data['Volume'][i]:
            main_df['Volume'][i] = main_df['Volume'][i].replace('M', '')
            main_df['Volume'][i] = float(main_df['Volume'][i]) * 1000000
        else:
            main_df['Volume'][i] = float(main_df['Volume'][i])
    except:
        print(data['date'][i])

main_df["Volume"] = main_df["Volume"].astype(float)
main_df["Close"] = main_df["Close"].str.replace(',', '').astype(float)
main_df["Open"] = main_df["Open"].str.replace(',', '').astype(float)
main_df["High"] = main_df["High"].str.replace(',', '').astype(float)
main_df["Low"] = main_df["Low"].str.replace(',', '').astype(float)
main_df.tail(10)

,Close,Open,High,Low,Volume
Date,,,,,
2023-07-10,77900.0,75400.0,77900.0,75400.0,2350000.0
2023-07-11,79200.0,78100.0,80100.0,78100.0,2850000.0
2023-07-12,79500.0,80100.0,80100.0,79000.0,1610000.0
2023-07-13,81800.0,79600.0,81900.0,79500.0,2670000.0
2023-07-14,81800.0,82000.0,82700.0,80700.0,1690000.0
2023-07-17,80700.0,81800.0,81800.0,80500.0,1450000.0
2023-07-18,80500.0,80600.0,81300.0,80200.0,1270000.0
2023-07-19,80800.0,80600.0,81300.0,80500.0,1370000.0
2023-07-20,79900.0,80900.0,80900.0,79100.0,2200000.0


In [92]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2492 entries, 2013-07-29 to 2023-07-21
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2492 non-null   float64
 1   Open    2492 non-null   float64
 2   High    2492 non-null   float64
 3   Low     2492 non-null   float64
 4   Volume  2492 non-null   float64
dtypes: float64(5)
memory usage: 116.8 KB


In [93]:
#Create the technical indicator by CustomStrategy

main_df.ta.strategy(CustomStrategy)
main_df.head(10)

,Close,Open,High,Low,Volume,NATR_3,RSI_3,ADX_3,DMP_3,DMN_3,...,WILLR_3,OBV,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_3_2.0,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0
Date,,,,,,,,,,,,,,,,,,,,,
2013-07-29,88500.0,90000.0,91000.0,88000.0,155990.0,NaN,NaN,NaN,NaN,NaN,...,NaN,155990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-30,89000.0,88000.0,89500.0,88000.0,156250.0,NaN,NaN,NaN,NaN,NaN,...,NaN,312240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-31,88500.0,89000.0,89500.0,88000.0,79700.0,1.694915,NaN,NaN,NaN,NaN,...,-83.333333,232540.0,NaN,NaN,NaN,88195.262146,88666.666667,89138.071187,1.063318,0.323223
2013-08-01,88000.0,88500.0,88500.0,88000.0,273930.0,1.136364,21.052632,NaN,0.000000,0.000000,...,-100.000000,-41390.0,NaN,NaN,NaN,87683.503419,88500.000000,89316.496581,1.845190,0.193814
2013-08-02,88500.0,88500.0,89000.0,88000.0,128920.0,1.129944,53.846154,NaN,20.454545,0.000000,...,-66.666667,87530.0,NaN,NaN,NaN,87861.928813,88333.333333,88804.737854,1.067331,0.676777
2013-08-05,88000.0,88500.0,89000.0,88000.0,87000.0,1.136364,33.175355,NaN,12.676056,0.000000,...,-100.000000,530.0,NaN,NaN,NaN,87695.262146,88166.666667,88638.071187,1.069349,0.323223
2013-08-06,87500.0,87500.0,88000.0,86500.0,291510.0,1.428571,21.052632,87.775891,6.831120,46.110057,...,-60.000000,-290980.0,NaN,NaN,NaN,87183.503419,88000.000000,88816.496581,1.855674,0.193814
2013-08-07,88000.0,87500.0,88500.0,87000.0,200230.0,1.562500,49.004371,60.182898,17.666854,27.257431,...,-40.000000,-90750.0,NaN,NaN,NaN,87361.928813,87833.333333,88304.737854,1.073407,0.676777
2013-08-08,88500.0,88000.0,92000.0,88000.0,235810.0,3.036723,66.693101,64.338011,61.002341,10.342626,...,-63.636364,145060.0,NaN,NaN,NaN,87183.503419,88000.000000,88816.496581,1.855674,0.806186


In [94]:
#Create the technical indicator by create_tema function (2, 8, 100)

main_df['tema_2'] = create_tema(main_df, 2)
main_df['tema_8'] = create_tema(main_df, 8)
main_df['tema_100'] = create_tema(main_df, 100)
main_df.head(10)

,Close,Open,High,Low,Volume,NATR_3,RSI_3,ADX_3,DMP_3,DMN_3,...,MACDh_12_26_9,MACDs_12_26_9,BBL_3_2.0,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,tema_2,tema_8,tema_100
Date,,,,,,,,,,,,,,,,,,,,,
2013-07-29,88500.0,90000.0,91000.0,88000.0,155990.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-30,89000.0,88000.0,89500.0,88000.0,156250.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88750.000000,NaN,NaN
2013-07-31,88500.0,89000.0,89500.0,88000.0,79700.0,1.694915,NaN,NaN,NaN,NaN,...,NaN,NaN,88195.262146,88666.666667,89138.071187,1.063318,0.323223,88509.259259,NaN,NaN
2013-08-01,88000.0,88500.0,88500.0,88000.0,273930.0,1.136364,21.052632,NaN,0.000000,0.000000,...,NaN,NaN,87683.503419,88500.000000,89316.496581,1.845190,0.193814,88009.259259,NaN,NaN
2013-08-02,88500.0,88500.0,89000.0,88000.0,128920.0,1.129944,53.846154,NaN,20.454545,0.000000,...,NaN,NaN,87861.928813,88333.333333,88804.737854,1.067331,0.676777,88459.876543,NaN,NaN
2013-08-05,88000.0,88500.0,89000.0,88000.0,87000.0,1.136364,33.175355,NaN,12.676056,0.000000,...,NaN,NaN,87695.262146,88166.666667,88638.071187,1.069349,0.323223,88031.207133,NaN,NaN
2013-08-06,87500.0,87500.0,88000.0,86500.0,291510.0,1.428571,21.052632,87.775891,6.831120,46.110057,...,NaN,NaN,87183.503419,88000.000000,88816.496581,1.855674,0.193814,87507.887517,NaN,NaN
2013-08-07,88000.0,87500.0,88500.0,87000.0,200230.0,1.562500,49.004371,60.182898,17.666854,27.257431,...,NaN,NaN,87361.928813,87833.333333,88304.737854,1.073407,0.676777,87958.962048,88250.000000,NaN
2013-08-08,88500.0,88000.0,92000.0,88000.0,235810.0,3.036723,66.693101,64.338011,61.002341,10.342626,...,NaN,NaN,87183.503419,88000.000000,88816.496581,1.855674,0.806186,88494.525733,88382.373114,NaN


In [95]:
#Check null value in the dataframe

main_df.isnull().sum()

Close             0
Open              0
High              0
Low               0
Volume            0
NATR_3            2
RSI_3             3
ADX_3             6
DMP_3             3
DMN_3             3
CCI_3_0.015      10
ROC_3             3
STOCHk_14_3_3    15
STOCHd_14_3_3    17
WILLR_3           2
OBV               0
MACD_12_26_9     25
MACDh_12_26_9    33
MACDs_12_26_9    33
BBL_3_2.0         2
BBM_3_2.0         2
BBU_3_2.0         2
BBB_3_2.0         2
BBP_3_2.0         2
tema_2            1
tema_8            7
tema_100         99
dtype: int64

In [96]:
#Drop null value in the dataframe

main_df = main_df.dropna()
main_df.isnull().sum()

Close            0
Open             0
High             0
Low              0
Volume           0
NATR_3           0
RSI_3            0
ADX_3            0
DMP_3            0
DMN_3            0
CCI_3_0.015      0
ROC_3            0
STOCHk_14_3_3    0
STOCHd_14_3_3    0
WILLR_3          0
OBV              0
MACD_12_26_9     0
MACDh_12_26_9    0
MACDs_12_26_9    0
BBL_3_2.0        0
BBM_3_2.0        0
BBU_3_2.0        0
BBB_3_2.0        0
BBP_3_2.0        0
tema_2           0
tema_8           0
tema_100         0
dtype: int64

In [97]:
#Check the length of the dataframe

len(main_df)

2387

In [98]:
#Export CSV file

main_df.to_csv('/mnt/learning/last_project/fetch_data_stock/input/clean_' + the_name_of_stock + '.csv')